<a href="https://colab.research.google.com/github/imammarzuki/Eksperimen/blob/main/CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contoh Tutorial dari ChatGPT 4

In [4]:
import pandas as pd
from sklearn.metrics import confusion_matrix, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np
import math

# Contoh dataset tentang destinasi wisata di Indonesia dengan pengguna dan rating (multi-kriteria)
# Setiap baris mewakili satu pengguna yang memberi rating pada beberapa destinasi
data = {
    'user': ['User1', 'User1', 'User1', 'User2', 'User2', 'User3', 'User3', 'User4', 'User4', 'User4'],
    'destination': ['Bali', 'Lombok', 'Yogyakarta', 'Bali', 'Yogyakarta', 'Bali', 'Bandung', 'Lombok', 'Bandung', 'Raja Ampat'],
    'rating': [4, 5, 4, 3, 5, 5, 3, 4, 2, 5],
    'price_sensitivity': [3, 4, 2, 1, 3, 4, 2, 4, 1, 5],  # Contoh kriteria tambahan
    'popularity': [5, 4, 3, 4, 3, 5, 2, 4, 1, 5]
}

# Membuat DataFrame
df = pd.DataFrame(data)

# Memutar tabel untuk membuat matriks pivot user-item
pivot = df.pivot_table(index='user', columns='destination', values='rating').fillna(0)

# Menyiapkan model Collaborative Filtering berbasis K-Nearest Neighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(pivot)

# Fungsi untuk membuat rekomendasi berdasarkan pengguna yang serupa
def get_recommendations(user):
    if user not in pivot.index:
        return f"User '{user}' tidak ditemukan dalam dataset."

    distances, indices = model_knn.kneighbors(pivot.loc[user].values.reshape(1, -1), n_neighbors=3)
    nearest_users = [pivot.index[i] for i in indices.flatten()]

    # Menggabungkan preferensi pengguna serupa untuk membuat rekomendasi
    recommendations = pivot.loc[nearest_users].mean(axis=0)
    recommendations = recommendations[recommendations > 0].sort_values(ascending=False)

    return recommendations.head()

# Contoh penggunaan
user_to_recommend = 'User1'
recommendations = get_recommendations(user_to_recommend)
print(f"Rekomendasi untuk '{user_to_recommend}':")
print(recommendations)

# Evaluasi menggunakan MAE dan RMSE
y_true = df['rating']
y_pred = np.mean(df['rating'])  # Prediksi rata-rata

# Menghitung MAE dan RMSE
mae = mean_absolute_error(y_true, np.full_like(y_true, y_pred))
rmse = math.sqrt(mean_squared_error(y_true, np.full_like(y_true, y_pred)))

print("\nMean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

# Contoh confusion matrix (asumsikan tujuan biner, misalnya 1 = 'direkomendasikan', 0 = 'tidak direkomendasikan')
y_true_binary = np.where(df['rating'] >= 3, 1, 0)
y_pred_binary = np.where(y_pred >= 3, 1, 0)
conf_matrix = confusion_matrix(y_true_binary, np.full_like(y_true_binary, y_pred_binary))

print("\nConfusion Matrix:")
print(conf_matrix)

Rekomendasi untuk 'User1':
destination
Bali          4.000000
Yogyakarta    3.000000
Lombok        1.666667
Bandung       1.000000
dtype: float64

Mean Absolute Error (MAE): 0.8
Root Mean Squared Error (RMSE): 1.0

Confusion Matrix:
[[0 1]
 [0 9]]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


# Mencoba dengan Dataset Kaggle

In [5]:
event=pd.read_csv("eco_event.csv")
place=pd.read_csv("eco_place.csv")
rating=pd.read_csv("eco_rating.csv")

In [6]:
event.head()

,event_id,event_img,event_name,event_place,event_date,event_about
0,1,https://storage.googleapis.com/travelee-capsto...,FESTIVAL ECOTOURISM 2022 - Membangun Investasi...,"Manokwari City Mall (MCM), Papua Barat",29 September - 1 Oktober 2022,Ayo bergabung dalam FESTIVAL ECOTOURISM 2022 R...
1,2,https://storage.googleapis.com/travelee-capsto...,Festival Taman nasional Taman Wisata Alam Indo...,"Lapangan Aldiron, Jl. Gatot Subroto, Jakarta",5 - 7 Agustus 2022,Sambut Festival FTNTWA dan Festival Ekowisata ...
2,3,https://storage.googleapis.com/travelee-capsto...,Medan - Belawan Mangrove Forest Festival,"Mako Lantamal-I, Medan, Sumatra Utara",19 Agustus 2018,Sambut keindahan hutan mangrove di Medan - Bel...
3,4,https://storage.googleapis.com/travelee-capsto...,FESTIVAL MANGROVE 2022,"Kuala Langsa, Aceh",10 Desember 2022,Selamat datang di Festival Mangrove 2022. Raya...
4,5,https://storage.googleapis.com/travelee-capsto...,INDOFEST INDONESIA OUTDOOR FESTIVAL 2023,"Istora Senayan, Jakarta Selatan",1 - 4 Juni 2023,Selamat datang di Indofest Indonesia Outdoor F...


In [7]:
place.head()

,place_id,place_name,place_description,category,city,price,rating,description_location,place_img,gallery_photo_img1,gallery_photo_img2,gallery_photo_img3,place_map
0,1,Taman Nasional Gunung Leuser,Taman Nasional Gunung Leuser adalah salah satu...,"Budaya,Taman Nasional",Aceh,25000,4.5,"Barisan mountain range, Aceh 24653",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Taman+Nasio...
1,2,Desa Wisata Munduk,Desa Wisata Munduk adalah sebuah desa di pegun...,Desa Wisata,Bali,10000,4.5,"Munduk, Banjar, Kabupaten Buleleng, Bali",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://goo.gl/maps/LyeJ2mAeFGysTE9v9
2,3,Desa Wisata Penglipuran,Desa Wisata Penglipuran adalah sebuah desa wis...,"Budaya,Desa Wisata",Bali,25000,4.8,"Jl. Penglipuran, Kubu, Kec. Bangli, Kabupaten ...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Desa+Wisata...
3,4,Taman Nasional Bali Barat,Taman Nasional Bali Barat adalah kawasan konse...,"Taman Nasional,Cagar Alam",Bali,15000,4.5,"Sumber Klampok, Bali",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Taman+Nasio...
4,5,Bukit Jamur,Bukit Jamur Ciwidey adalah satu dari sekian ba...,Cagar Alam,Bandung,12000,4.2,"Sugihmukti, Kec. Pasirjambu, Kabupaten Bandung...",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://www.google.com/maps/search/Bukit+Jamur


In [8]:
rating.head()

,user_id,place_id,user_rating
0,1,28,2
1,1,86,4
2,1,94,4
3,1,146,3
4,2,17,2


In [9]:
# Hanya ada dua dataframe yang digunakan yaitu place dan rating maka keduanya harus di merger

In [11]:
data=place.merge(rating,on="place_id")

In [12]:
data.head()

,place_id,place_name,place_description,category,city,price,rating,description_location,place_img,gallery_photo_img1,gallery_photo_img2,gallery_photo_img3,place_map,user_id,user_rating
0,2,Desa Wisata Munduk,Desa Wisata Munduk adalah sebuah desa di pegun...,Desa Wisata,Bali,10000,4.5,"Munduk, Banjar, Kabupaten Buleleng, Bali",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://goo.gl/maps/LyeJ2mAeFGysTE9v9,46,5
1,2,Desa Wisata Munduk,Desa Wisata Munduk adalah sebuah desa di pegun...,Desa Wisata,Bali,10000,4.5,"Munduk, Banjar, Kabupaten Buleleng, Bali",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://goo.gl/maps/LyeJ2mAeFGysTE9v9,75,3
2,2,Desa Wisata Munduk,Desa Wisata Munduk adalah sebuah desa di pegun...,Desa Wisata,Bali,10000,4.5,"Munduk, Banjar, Kabupaten Buleleng, Bali",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://goo.gl/maps/LyeJ2mAeFGysTE9v9,79,4
3,2,Desa Wisata Munduk,Desa Wisata Munduk adalah sebuah desa di pegun...,Desa Wisata,Bali,10000,4.5,"Munduk, Banjar, Kabupaten Buleleng, Bali",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://goo.gl/maps/LyeJ2mAeFGysTE9v9,94,4
4,2,Desa Wisata Munduk,Desa Wisata Munduk adalah sebuah desa di pegun...,Desa Wisata,Bali,10000,4.5,"Munduk, Banjar, Kabupaten Buleleng, Bali",https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://storage.googleapis.com/travelee-capsto...,https://goo.gl/maps/LyeJ2mAeFGysTE9v9,137,3


In [13]:
data.columns

Index(['place_id', 'place_name', 'place_description', 'category', 'city',
       'price', 'rating', 'description_location', 'place_img',
       'gallery_photo_img1', 'gallery_photo_img2', 'gallery_photo_img3',
       'place_map', 'user_id', 'user_rating'],
      dtype='object')

In [14]:
# Hanya kolom user, destination, rating, place yang digunakan

In [15]:
data2=data[["user_id","place_name","rating","price"]]
data2.head()

,user_id,place_name,rating,price
0,46,Desa Wisata Munduk,4.5,10000
1,75,Desa Wisata Munduk,4.5,10000
2,79,Desa Wisata Munduk,4.5,10000
3,94,Desa Wisata Munduk,4.5,10000
4,137,Desa Wisata Munduk,4.5,10000


In [16]:
df=data2.rename({"user_id":"user","place_name":"destination"},axis=1)

In [19]:
df

,user,destination,rating,price
0,46,Desa Wisata Munduk,4.5,10000
1,75,Desa Wisata Munduk,4.5,10000
2,79,Desa Wisata Munduk,4.5,10000
3,94,Desa Wisata Munduk,4.5,10000
4,137,Desa Wisata Munduk,4.5,10000
...,...,...,...,...
841,85,Wisata Kaliurang,4.4,8000
842,86,Wisata Kaliurang,4.4,8000
843,101,Wisata Kaliurang,4.4,8000
844,102,Wisata Kaliurang,4.4,8000


In [20]:
# Memutar tabel untuk membuat matriks pivot user-item
pivot = df.pivot_table(index='user', columns='destination', values='rating').fillna(0)

# Menyiapkan model Collaborative Filtering berbasis K-Nearest Neighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(pivot)

# Fungsi untuk membuat rekomendasi berdasarkan pengguna yang serupa
def get_recommendations(user):
    if user not in pivot.index:
        return f"User '{user}' tidak ditemukan dalam dataset."

    distances, indices = model_knn.kneighbors(pivot.loc[user].values.reshape(1, -1), n_neighbors=3)
    nearest_users = [pivot.index[i] for i in indices.flatten()]

    # Menggabungkan preferensi pengguna serupa untuk membuat rekomendasi
    recommendations = pivot.loc[nearest_users].mean(axis=0)
    recommendations = recommendations[recommendations > 0].sort_values(ascending=False)

    return recommendations.head()

# Contoh penggunaan
user_to_recommend = 46
recommendations = get_recommendations(user_to_recommend)
print(f"Rekomendasi untuk '{user_to_recommend}':")
print(recommendations)

# Evaluasi menggunakan MAE dan RMSE
y_true = df['rating']
y_pred = np.mean(df['rating'])  # Prediksi rata-rata

# Menghitung MAE dan RMSE
mae = mean_absolute_error(y_true, np.full_like(y_true, y_pred))
rmse = math.sqrt(mean_squared_error(y_true, np.full_like(y_true, y_pred)))

print("\nMean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

# Contoh confusion matrix (asumsikan tujuan biner, misalnya 1 = 'direkomendasikan', 0 = 'tidak direkomendasikan')
y_true_binary = np.where(df['rating'] >= 3, 1, 0)
y_pred_binary = np.where(y_pred >= 3, 1, 0)
conf_matrix = confusion_matrix(y_true_binary, np.full_like(y_true_binary, y_pred_binary))

print("\nConfusion Matrix:")
print(conf_matrix)

Rekomendasi untuk '46':
destination
Taman Nasional Ujung Kulon       4.400000
Curug Luhur Waterfall            4.200000
Desa Wisata Mangunan             1.566667
Taman Nasional Gunung Merbabu    1.566667
Seribu Batu Songgo Langit        1.533333
dtype: float64

Mean Absolute Error (MAE): 0.16874793912669261
Root Mean Squared Error (RMSE): 0.2208361784669299

Confusion Matrix:
[[846]]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


# Menggunakan Data Wisata Batu

In [21]:
data=pd.read_excel("Dataset_227(10F).xlsx")
data.head()

,jenis_kelamin,umur,pekerjaan,hobi,tujuan_berwisata,status_perkawinan,daerah_asal,teman_perjalanan,pendidikan_terakhir,repetition,...,Y5,Y6,Y7,Y8,Y9,Y10,Y11,Y12,Y13,Y14
0,Laki-Laki,12-25 tahun,Pelajar/Mahasiswa,Olahraga,Rekreasi/Liburan,Belum Menikah,Luar Kota Batu,Pasangan,SMA/SMK,Pernah,...,1,0,0,0,0,0,0,0,0,0
1,Perempuan,12-25 tahun,Wiraswasta,Traveling,Rekreasi/Liburan,Belum Menikah,Luar Kota Batu,Teman,SMA/SMK,Belum Pernah,...,1,0,0,1,0,0,0,0,0,0
2,Perempuan,12-25 tahun,Pelajar/Mahasiswa,Traveling,Rekreasi/Liburan,Belum Menikah,Luar Kota Batu,Keluarga,SMA/SMK,Pernah,...,1,1,0,1,0,0,0,1,0,0
3,Perempuan,12-25 tahun,Pelajar/Mahasiswa,Traveling,Rekreasi/Liburan,Belum Menikah,Luar Kota Batu,Teman,SMA/SMK,Belum Pernah,...,1,1,0,1,0,1,0,0,0,0
4,Laki-Laki,12-25 tahun,Pelajar/Mahasiswa,Olahraga,Rekreasi/Liburan,Belum Menikah,Luar Kota Batu,Teman,SMA/SMK,Pernah,...,0,1,1,1,0,0,0,0,0,0
